# Foundation 00: PostgreSQL Schema Setup

**What This Notebook Does:**
- Creates the database schema for the entire RAG learning system
- **RUN THIS FIRST** before foundation/02, intermediate, advanced, or evaluation notebooks

**Expected Runtime:** 1-2 minutes

**Prerequisites:**
1. PostgreSQL running (via Docker or locally)
2. psycopg2-binary installed (`pip install psycopg2-binary`)
3. The `pgvector` extension (automatically created when you start the PostgreSQL container)

---

## The RAG System Architecture

This notebook creates 4 core tables that power the entire learning system:

| Table | Purpose | Who Uses It |
|-------|---------|-----------|
| **embedding_registry** | Catalog of embedding models with metadata | foundation/02, intermediate/03-04, advanced/05-10 |
| **evaluation_groundtruth** | Human-curated test questions | evaluation-lab/01-04 |
| **experiments** | Track each technique/comparison run | All advanced notebooks |
| **evaluation_results** | Metrics from each experiment | evaluation-lab/02-04 |

**Why this structure?**
- **Reuse embeddings**: Don't regenerate (50+ minutes) - just load from registry
- **Compare techniques**: Run multiple experiments with same embeddings, different configurations
- **Track progress**: See how each technique (reranking, query expansion, etc.) improves metrics
- **Share results**: Export experiments and metrics for documentation

---

## Prerequisites

In [ ]:
import psycopg2
import json
from datetime import datetime

## Configuration

In [ ]:
# PostgreSQL connection
POSTGRES_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'rag_db',
    'user': 'postgres',
    'password': 'postgres',
}

## Connect to Database

In [ ]:
try:
    conn = psycopg2.connect(
        host=POSTGRES_CONFIG['host'],
        port=POSTGRES_CONFIG['port'],
        database=POSTGRES_CONFIG['database'],
        user=POSTGRES_CONFIG['user'],
        password=POSTGRES_CONFIG['password']
    )
    print(f"✓ Connected to PostgreSQL at {POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}")
except psycopg2.OperationalError as e:
    print(f"✗ Failed to connect to PostgreSQL: {e}")
    print("Make sure PostgreSQL is running. Start with:")
    print("docker run -d --name pgvector-rag \\")
    print("  -e POSTGRES_PASSWORD=postgres -e POSTGRES_DB=rag_db \\")
    print("  -p 5432:5432 -v pgvector_data:/var/lib/postgresql/data \\")
    print("  pgvector/pgvector:pg16")
    raise

## Create Schema Tables

The following cells create the 4 core tables with proper constraints and metadata storage.

In [ ]:
# CREATE TABLE 1: embedding_registry
# Purpose: Catalog of embedding models so we can reuse them across notebooks
# Used by: foundation/02 (register), intermediate/03 (discover), all advanced techniques (load)

with conn.cursor() as cur:
    cur.execute('''
        CREATE TABLE IF NOT EXISTS embedding_registry (
            id SERIAL PRIMARY KEY,
            model_alias TEXT UNIQUE NOT NULL,        -- Short name like 'bge_base_en_v1_5'
            model_name TEXT NOT NULL,                -- Full name like 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
            dimension INT NOT NULL,                  -- Vector size (768 for BGE)
            embedding_count INT DEFAULT 0,           -- How many chunks embedded
            chunk_source_dataset TEXT,               -- Where chunks came from ('Wikipedia 10MB')
            chunk_size_config INT,                   -- Max chunk size used (1000 chars)
            metadata_json JSONB DEFAULT '{}'::jsonb, -- Flexible: preserve_existing, timestamp, etc.
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            last_accessed TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    print("✓ Created embedding_registry table")
    print("  Purpose: Catalog of embedding models for reuse across notebooks")
    print("  Example: Store 'bge_base_en_v1_5' with 768 dimensions, 1000+ chunks")

In [ ]:
# CREATE TABLE 2: evaluation_groundtruth
# Purpose: Curated test questions for measuring RAG quality
# Used by: evaluation-lab/01 (create/manage), evaluation-lab/02-04 (compute metrics)

with conn.cursor() as cur:
    cur.execute('''
        CREATE TABLE IF NOT EXISTS evaluation_groundtruth (
            id SERIAL PRIMARY KEY,
            question TEXT NOT NULL,                          -- "What is photosynthesis?"
            source_type TEXT CHECK (source_type IN ('llm_generated', 'template_based', 'manual')),
            relevant_chunk_ids INT ARRAY,                   -- Which chunks answer this? [42, 157, 203]
            quality_rating TEXT CHECK (quality_rating IN ('good', 'bad', 'ambiguous', 'rejected')),
            human_notes TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            created_by TEXT                                 -- Who created/curated it?
        )
    ''')
    conn.commit()
    print("✓ Created evaluation_groundtruth table")
    print("  Purpose: Curated test set for measuring retrieval/generation quality")
    print("  Example: Store 'What is photosynthesis?' with relevant chunk IDs")

In [ ]:
# CREATE TABLE 3: experiments
# Purpose: Track each technique/configuration run for reproducibility
# Used by: All advanced notebooks (start_experiment), evaluation-lab (compare_experiments)

with conn.cursor() as cur:
    cur.execute('''
        CREATE TABLE IF NOT EXISTS experiments (
            id SERIAL PRIMARY KEY,
            experiment_name TEXT NOT NULL,                  -- "Reranking with CrossEncoder"
            notebook_path TEXT,                             -- "advanced-techniques/05-reranking.ipynb"
            embedding_model_alias TEXT,                     -- What embeddings used? 'bge_base_en_v1_5'
            config_hash TEXT,                               -- SHA256 hash of exact config (for reproducibility)
            config_json JSONB,                              -- Full config: top_n, rerank_threshold, etc.
            techniques_applied TEXT ARRAY DEFAULT '{}'::text[],  -- ['reranking', 'hybrid_search']
            started_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            completed_at TIMESTAMP,
            status TEXT DEFAULT 'running' CHECK (status IN ('running', 'completed', 'failed')),
            notes TEXT,
            FOREIGN KEY (embedding_model_alias) REFERENCES embedding_registry(model_alias)
        )
    ''')
    conn.commit()
    print("✓ Created experiments table")
    print("  Purpose: Track each technique run with full config for reproducibility")
    print("  Example: Store reranking experiment with config_hash to find all similar runs")

In [ ]:
# CREATE TABLE 4: evaluation_results
# Purpose: Store metrics from each experiment run
# Used by: All advanced notebooks (save_metrics), evaluation-lab/02-04 (query & visualize)

with conn.cursor() as cur:
    cur.execute('''
        CREATE TABLE IF NOT EXISTS evaluation_results (
            id SERIAL PRIMARY KEY,
            experiment_id INT NOT NULL,                     -- Links to experiments table
            metric_name TEXT NOT NULL,                      -- 'Precision@5', 'NDCG', 'BLEU'
            metric_value FLOAT NOT NULL,                    -- 0.75 (75% precision)
            metric_details_json JSONB DEFAULT '{}'::jsonb, -- Flexible: per-query breakdown, etc.
            computed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (experiment_id) REFERENCES experiments(id) ON DELETE CASCADE
        )
    ''')
    conn.commit()
    print("✓ Created evaluation_results table")
    print("  Purpose: Store metrics computed for each experiment")
    print("  Example: Store Precision@5=0.75 for experiment 42")

## Create Indexes for Query Performance

In [ ]:
# Create indexes for common queries
indexes = [
    "CREATE INDEX IF NOT EXISTS idx_experiments_embedding_model ON experiments(embedding_model_alias)",
    "CREATE INDEX IF NOT EXISTS idx_experiments_status ON experiments(status)",
    "CREATE INDEX IF NOT EXISTS idx_experiments_started ON experiments(started_at DESC)",
    "CREATE INDEX IF NOT EXISTS idx_results_experiment ON evaluation_results(experiment_id)",
    "CREATE INDEX IF NOT EXISTS idx_results_metric ON evaluation_results(metric_name)",
    "CREATE INDEX IF NOT EXISTS idx_groundtruth_quality ON evaluation_groundtruth(quality_rating)",
]

with conn.cursor() as cur:
    for idx in indexes:
        cur.execute(idx)
    conn.commit()
    print(f"✓ Created {len(indexes)} indexes")

## Verify Schema Creation

In [ ]:
# Verify all tables were created
with conn.cursor() as cur:
    cur.execute('''
        SELECT table_name FROM information_schema.tables 
        WHERE table_schema = 'public' 
        AND table_name IN ('embedding_registry', 'evaluation_groundtruth', 'experiments', 'evaluation_results')
        ORDER BY table_name
    ''')
    tables = cur.fetchall()
    print(f"✓ Schema creation complete. Tables found:")
    for (table_name,) in tables:
        print(f"  - {table_name}")

conn.close()
print("\n✓ Database connection closed. Ready to proceed!")